**PRÁCTICA**: Buscad las diferencias con la DC GAN en escala de grises. En 10 minutos las ponemos en común y ejecutamos paso a paso para ver los resultados.

In [ ]:
# importamos las librerías necesarias
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
# configuramos Colab para que nos muestre las imágenes más grandes
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
# definimos el discriminador
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample (por el atributo 'strides')
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# clasificador
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compilamos modelo
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
# definimos el generador
def define_generator(latent_dim):
	model = Sequential()
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample a 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample a 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample a 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# salida (nuestra imagen fake)
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [ ]:
# definimos el modelo GAN combinando generador y discriminador, para entrenar el generador
def define_gan(g_model, d_model):
    # Así que congelamos el discriminador:
    d_model.trainable = False
    # ahora conectamos el G(z) al D(x)
    model = Sequential()
    # añadimos el generador primero: él es el encargado de generar una muestra
    # a partir del espacio latente
    model.add(g_model)
    # y el discriminador después: le introducimos la muestra generada por el 
    # G(z) para que nos diga si cree que es real o fake
    model.add(d_model)
    # y ahora sí, compilamos el modelo
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5)) 
    return model

In [ ]:
# definimos las funciones para cargar el MNIST
def load_real_samples():
    # cargamos el cifar10
    (trainX, _), (_, _) = load_data()
    # convertimos a float32
    X = trainX.astype('float32')
    # escalamos entre -1 y 1
    X = (X - 127.5) / 127.5
    return X

# nos creamos una función que nos devuelva n_samples del dataset con sus 
# etiquetas (1) 
def generate_real_samples(dataset, n_samples):
    # seleccionamos n_samples muestras aleatoriamente
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # las cogemos
    X = dataset[ix]
    # generamos las etiquetas reales (1)
    y = np.ones((n_samples, 1))
    return X, y

In [ ]:
# generamos los vectores latentes que introduciremos al generador
def generate_latent_points(latent_dim, batch_size):
    # generamos un vector de batch_size * latent_dim números aleatorios
    # latent_dim es la dimensión del vector latente
    # batch_size es el número de elementos por batch
    x_input = np.random.randn(latent_dim * batch_size)
    # redimensionamos el vector para que tenga un tamaño (batch_size, latent_dim)
    x_input = x_input.reshape(batch_size, latent_dim)
    return x_input

# creamos datos fake con el generador (dinero falsificado)
def generate_fake_samples(g_model, latent_dim, n_samples): 
    # usamos la función anterior para generar los vectores latentes que 
    # necesitamos para generar muestras fake
    x_input = generate_latent_points(latent_dim, n_samples)
    # le introducimos los vectores latentes al generador para obtener
    # muestras similares a las reales
    X = g_model.predict(x_input)
    # le asignamos la etiqueta 0 (porque utilizaremos esta función para
    # entrenar el D)
    y = np.zeros((n_samples, 1)) 
    return X, y

In [ ]:
# función para guardar las imágenes generadas
def save_plot(examples, epoch, n=7):
	# escalamos de [-1,1] (la salida de nuestra gan, debido a la función de activación tanh) a [0,1]
	examples = (examples + 1) / 2.0
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i])
	# guardamos las imágenes
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()

In [ ]:
# función para entrenar la GAN: el discriminador y el generador
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # bucle para las epochs
    for epoch in range(n_epochs):
        # bucle para los batch
        for batch in range(bat_per_epo):
            
            # en esta ocasión vamos a separar las pérdidas del discriminador
            # cuando le metemos imágenes reales y cuando le metemos imágenes
            # fake para ver cómo lo hace con cada tipo
            # recordad que lo ideal es que llegue a un 50% de acc en cada uno

            # preparamos los datos reales
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # actualizamos el discriminador
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            
            # generamos datos falsos
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # actualizamos el discriminador
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			
            # preparamos los puntos en el espacio latente: serán la entrada al
            # modelo GAN con el que entrenaremos el generador
            X_gan = generate_latent_points(latent_dim, n_batch)
            
            # creamos etiquetas invertidas para el generador: utilizamos el D(x) 
            # para que piense que las muestras que le introducimos son reales, y
            # en caso de que diga que no son reales, aprovechamos la información
            # de sus gradientes para actualizar el G(z) para que la próxima vez
            # los datos generados por G(z) sean más plausibles (parecidos a los 
            # reales)
            y_gan = np.ones((n_batch, 1))
            
            # como acabamos de ver, entrenamos el generador de forma que actualice
            # sus pesos usando los gradientes del discriminador
            # tened en cuenta que en este modelo (gan_model) el discriminador está
            # congelado, por lo que no se actualizan sus pesos: no queremos "untar"
            # a nuestro policía, lo que queremos es fabricar dinero más realista.
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            # mostramos el progreso
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % 
                  (epoch+1, batch+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (epoch+1) % 10 == 0 or epoch == 0:
            # preparamos ejemplos reales
            X_real, y_real = generate_real_samples(dataset, n_batch)
            # evaluamos el discriminador con datos reales
            _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
            # preparamos ejemplos fake
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
            # evaluamos el discriminador con datos fake
            _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
            # mostramos cómo de bueno es nuestro policía
            print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
            # guardamos las imágenes generadas
            save_plot(x_fake, epoch)
            # guardamos el generador para tenerlo disponible más tarde
            filename = 'generator_model_%03d.h5' % (epoch + 1)
            g_model.save(filename)

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=256)

In [ ]:
# veamos los archivos generados
ls

In [ ]:
# montamos la unidad drive donde tenemos los datos en la carpeta drive/My Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# los guardamos en nuestro drive para evitar tener que reejecutar cada vez
!mkdir drive/My\ Drive/5_dcgan_color_cifar10
!cp *gen* drive/My\ Drive/5_dcgan_color_cifar10/
!ls -lah drive/My\ Drive/5_dcgan_color_cifar10/

In [ ]:
plt.imshow(plt.imread('generated_plot_e001.png'))
plt.show()

In [ ]:
plt.imshow(plt.imread('generated_plot_e020.png'))
plt.show()

In [ ]:
# no se ve bien, vamos a generar imagenes con el generador guardado!

In [ ]:
def plot_results(imgs, n_rows, n_cols, epoch, h=32, w=32):
    imgs = (imgs + 1) / 2.0
    output_img = np.zeros((n_rows*h, n_cols*w, 3), np.float)
    k = 0
    for i in range(n_rows):
        for j in range(n_cols):
            output_img[h*i:h*(i+1), w*j:w*(j+1), :] = imgs[k]
            k += 1
    plt.imshow(np.asarray(output_img*255., np.uint8))

In [ ]:
import numpy as np
latent_dim = 100
n_samples = 64

# definimos el mismo código latente para todos los tests
x_input = generate_latent_points(latent_dim, n_samples)

In [ ]:
g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_001.h5')
X = g_model.predict(x_input)

In [ ]:
X.shape

In [ ]:
plot_results(X, 8, 8, 10)

In [ ]:
# veamos el modelo 20
g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_020.h5')
X = g_model.predict(x_input)
plot_results(X, 8, 8, 100)

In [ ]:
# generamos imágenes diferentes con un código latente distinto
x_input = generate_latent_points(latent_dim, n_samples)

g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_020.h5')
X = g_model.predict(x_input)
plot_results(X, 8, 8, 200)

In [ ]:
# generamos imágenes diferentes con un código latente distinto
x_input = generate_latent_points(latent_dim, n_samples)

g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_020.h5')
X = g_model.predict(x_input)
plot_results(X, 8, 8, 200)

In [ ]:
# generamos imágenes diferentes con un código latente distinto
x_input = generate_latent_points(latent_dim, n_samples)

g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_020.h5')
X = g_model.predict(x_input)
plot_results(X, 8, 8, 200)